## Pydantic Tool Calling with the Converse API

The tool schema created by Pydantic does not directly match the schema expected by the Converse API. In this notebook, we will walk through how you can map pydantic tools to the expected schema. 

#### Setup

Install pydantic if needed

In [ ]:
%pip install pydantic

#### Tool Conversion

First we will start by creating a few basic Pydantic tools that can get information about the weather

In [26]:
from pydantic import BaseModel, Field
from typing import List


class GetCurrentWeather(BaseModel):
    "Use this function to get the realtime weather given a location"
    location: str = Field(description="location of the place")
    date: str = Field(description="the date")


class GetAverageTemperature(BaseModel):
    "Use this function to get the average temperature of a location"
    location: str = Field(description="location of the place")

We can view the default schema created by calling model_json_schema()

In [ ]:
schema = GetCurrentWeather.model_json_schema()
schema

Using the values provided from the default schema, we will generate a function that will programmatically convert the tool schemas to the format expected by the converse API

In [27]:
def pydantic_to_tool_use_conversion(tools: List[type[BaseModel]]):
    return {
        "tools": [
            {
                "toolSpec": {
                    "name": schema["title"],
                    "description": schema["description"],
                    "inputSchema": {
                        "json": {
                            "type": "object",
                            "properties": schema["properties"],
                            "required": schema["required"],
                        }
                    },
                }
            }
            for tool in tools
            if (schema := tool.model_json_schema())
        ]
    }

We can now use this method to use our pydantic tools when invoking the Converse API

In [34]:
import boto3
from datetime import date

LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"

client = boto3.client("bedrock-runtime", region_name="us-east-1")


def ask_about_the_weather(query: str):
    print(f"User: {query}")

    system = [
        {
            "text": f"""You are a helpful travel assistant. 
            You have access to external tools to help give recommendations to the user. 
            The current date is: {date.today()}"""
        }
    ]

    messages = [{"role": "user", "content": [{"text": query}]}]

    inf_params = {"maxTokens": 300, "topP": 1, "temperature": 1}

    resp = client.converse(
        modelId=LITE_MODEL_ID,
        system=system,
        messages=messages,
        inferenceConfig=inf_params,
        additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
        toolConfig=pydantic_to_tool_use_conversion(
            [GetCurrentWeather, GetAverageTemperature]
        ),
    )

    tool_use = next(
        block["toolUse"]
        for block in resp["output"]["message"]["content"]
        if "toolUse" in block
    )
    print(f"Tool: {tool_use}\n")

In [ ]:
ask_about_the_weather("What is the weather in San Fransisco?")
ask_about_the_weather("What is the average temperature in San Fransisco?")